<a href="https://colab.research.google.com/github/Natdata/Reading_calculator/blob/main/Kalkulator_przeczyta%C5%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
start = '2024-08-06'
end = '2024-08-15'
value = ['375994']
numbers_of_pages = 38


In [ ]:
from datetime import datetime, timedelta
import requests
import json
import pandas as pd
import time

event_name = 'leaflet_quit_full'
name = 'leaflet_id'
operator = 'equals'

start = start.replace('-', '')
start = int(start)

end = end.replace('-', '')
end = int(end)

headers = {
    'X-CleverTap-Account-Id': 'aaa',
    'X-CleverTap-Passcode': 'bbb',
    'Content-Type': 'application/json',
}

data = {
    "event_name": event_name,
    "event_properties": [{"name": name, "operator": operator, "value": value}],
    "from": start,
    "to": end
}

results = []  # Lista do przechowywania wyników
json_data = json.dumps(data)
response = requests.post('https://eu1.api.clevertap.com/1/counts/events.json', headers=headers, data=json_data)
response_json = response.json()

req_id = response_json['req_id']

    # Odczekanie 5 sekund przed wykonaniem kolejnego żądania GET
time.sleep(3)

response1 = requests.get(
        f'https://eu1.api.clevertap.com/1/counts/events.json?req_id={req_id}', headers=headers)
response1_json = response1.json()

    # Dodawanie wyników do listy jako słownik
result = {

        "Count": response1_json.get("count", 0)
    }
results.append(result)

# Tworzenie DataFrame z wyników
Leaflet_quit_full_count = pd.DataFrame(results)

print(Leaflet_quit_full_count)
print()





   Count
0   3602



In [ ]:
event_name2 = 'leaflet_quit_part'

results = []
page_range = 0

while page_range <= numbers_of_pages:
    print(f"Page range is now: '{page_range}'")
    data = {
        "event_name": event_name2,
        "event_properties": [
            {"name": name, "operator": operator, "value": value},
            {"name": "seen_pages_count", "operator": "equals", "value": page_range}
        ],
        "from": start,
        "to": end
    }
    json_data = json.dumps(data)
    print(f"Sending data: {json_data}")

    response = requests.post('https://eu1.api.clevertap.com/1/counts/events.json', headers=headers, data=json_data)

    # Obsługa błędów
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        print(response.text)
        break

    response_json = response.json()

    if 'req_id' in response_json:
        req_id = response_json['req_id']
        time.sleep(5)  # Dłuższa przerwa między żądaniami
        response1 = requests.get(f'https://eu1.api.clevertap.com/1/counts/events.json?req_id={req_id}', headers=headers)
        response1_json = response1.json()

        count = response1_json.get("count", 0)

        result = {
            "Page range": page_range,
            "Count": count
        }
        results.append(result)
        print(f"Result for page range {page_range}: {count}")

    page_range += 1

df = pd.DataFrame(results)
print("Final results:")
print(df)

Page range is now: '0'
Sending data: {"event_name": "leaflet_quit_part", "event_properties": [{"name": "leaflet_id", "operator": "equals", "value": ["375994"]}, {"name": "seen_pages_count", "operator": "equals", "value": 0}], "from": 20240806, "to": 20240815}
Result for page range 0: 0
Page range is now: '1'
Sending data: {"event_name": "leaflet_quit_part", "event_properties": [{"name": "leaflet_id", "operator": "equals", "value": ["375994"]}, {"name": "seen_pages_count", "operator": "equals", "value": 1}], "from": 20240806, "to": 20240815}
Result for page range 1: 2256
Page range is now: '2'
Sending data: {"event_name": "leaflet_quit_part", "event_properties": [{"name": "leaflet_id", "operator": "equals", "value": ["375994"]}, {"name": "seen_pages_count", "operator": "equals", "value": 2}], "from": 20240806, "to": 20240815}
Result for page range 2: 132
Page range is now: '3'
Sending data: {"event_name": "leaflet_quit_part", "event_properties": [{"name": "leaflet_id", "operator": "equa

In [ ]:
total_count_from_df = df['Count'].sum()

# Pobranie wartości 'Count' z Leaflet_quit_full_count
full_count = Leaflet_quit_full_count['Count'].iloc[0]

# Obliczenie zmiennej przeczytania
read_ratio = full_count / (total_count_from_df + full_count)
print(f"Zmienna przeczytania: {read_ratio * 100:.2f}%")
print(total_count_from_df)

Zmienna przeczytania: 34.84%
6737


In [ ]:
new_df = df.copy()

# Modyfikacja 'Count' w nowym DataFrame
for index, row in new_df.iterrows():
    if row['Page range'] == numbers_of_pages:
        new_df.at[index, 'Count'] = full_count
filtered_new_df = new_df[(new_df['Page range'] >= 1) & (new_df['Page range'] <= numbers_of_pages)]
# Wynik końcowy - nowy DataFrame
print(filtered_new_df)

    Page range  Count
1            1   2256
2            2    132
3            3    209
4            4     88
5            5     92
6            6     77
7            7     88
8            8    118
9            9     58
10          10     67
11          11     64
12          12     74
13          13     88
14          14     42
15          15     39
16          16     40
17          17     40
18          18     62
19          19     69
20          20     45
21          21     35
22          22     40
23          23     62
24          24     33
25          25     33
26          26     27
27          27     52
28          28     95
29          29    106
30          30     73
31          31     87
32          32     84
33          33    147
34          34    160
35          35    409
36          36    917
37          37    625
38          38   3602


In [ ]:
filtered_new_df['Cumulative Count'] = filtered_new_df['Count'][::-1].cumsum()[::-1]

# Tworzenie nowej tabeli z wykorzystaniem obliczonej kumulatywnej sumy
new_table = pd.DataFrame({
    'Page range': filtered_new_df['Page range'],
    'Count': filtered_new_df['Cumulative Count']
})

# Wynik końcowy - wyświetlenie nowej tabeli
print(new_table)

    Page range  Count
1            1  10335
2            2   8079
3            3   7947
4            4   7738
5            5   7650
6            6   7558
7            7   7481
8            8   7393
9            9   7275
10          10   7217
11          11   7150
12          12   7086
13          13   7012
14          14   6924
15          15   6882
16          16   6843
17          17   6803
18          18   6763
19          19   6701
20          20   6632
21          21   6587
22          22   6552
23          23   6512
24          24   6450
25          25   6417
26          26   6384
27          27   6357
28          28   6305
29          29   6210
30          30   6104
31          31   6031
32          32   5944
33          33   5860
34          34   5713
35          35   5553
36          36   5144
37          37   4227
38          38   3602


<ipython-input-38-5499f1c6256e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_new_df['Cumulative Count'] = filtered_new_df['Count'][::-1].cumsum()[::-1]


In [ ]:
first_count = new_table['Count'].iloc[0]  # Pierwsza wartość Count
new_table['Percentage'] = new_table['Count'] / first_count * 100

# Formatowanie kolumny Percentage do dwóch miejsc po przecinku
new_table['Percentage'] = new_table['Percentage'].map('{:.2f}%'.format)

# Wynik końcowy - wyświetlenie nowej tabeli z procentami
print(new_table[['Page range', 'Percentage']])

    Page range Percentage
1            1    100.00%
2            2     78.17%
3            3     76.89%
4            4     74.87%
5            5     74.02%
6            6     73.13%
7            7     72.39%
8            8     71.53%
9            9     70.39%
10          10     69.83%
11          11     69.18%
12          12     68.56%
13          13     67.85%
14          14     67.00%
15          15     66.59%
16          16     66.21%
17          17     65.82%
18          18     65.44%
19          19     64.84%
20          20     64.17%
21          21     63.73%
22          22     63.40%
23          23     63.01%
24          24     62.41%
25          25     62.09%
26          26     61.77%
27          27     61.51%
28          28     61.01%
29          29     60.09%
30          30     59.06%
31          31     58.36%
32          32     57.51%
33          33     56.70%
34          34     55.28%
35          35     53.73%
36          36     49.77%
37          37     40.90%
38          

In [ ]:
print(new_table[['Page range', 'Percentage']].to_string(index=False))

 Page range Percentage
          1    100.00%
          2     78.17%
          3     76.89%
          4     74.87%
          5     74.02%
          6     73.13%
          7     72.39%
          8     71.53%
          9     70.39%
         10     69.83%
         11     69.18%
         12     68.56%
         13     67.85%
         14     67.00%
         15     66.59%
         16     66.21%
         17     65.82%
         18     65.44%
         19     64.84%
         20     64.17%
         21     63.73%
         22     63.40%
         23     63.01%
         24     62.41%
         25     62.09%
         26     61.77%
         27     61.51%
         28     61.01%
         29     60.09%
         30     59.06%
         31     58.36%
         32     57.51%
         33     56.70%
         34     55.28%
         35     53.73%
         36     49.77%
         37     40.90%
         38     34.85%


In [ ]:
#Zamień mi kropki na przecinki w kolumnie Percentage
new_table['Percentage'] = new_table['Percentage'].str.replace('.', ',')

# Wynik końcowy - wyświetlenie nowej tabeli z przecinkami
print(new_table[['Page range','Percentage']].to_string(index=False))

 Page range Percentage
          1    100,00%
          2     78,17%
          3     76,89%
          4     74,87%
          5     74,02%
          6     73,13%
          7     72,39%
          8     71,53%
          9     70,39%
         10     69,83%
         11     69,18%
         12     68,56%
         13     67,85%
         14     67,00%
         15     66,59%
         16     66,21%
         17     65,82%
         18     65,44%
         19     64,84%
         20     64,17%
         21     63,73%
         22     63,40%
         23     63,01%
         24     62,41%
         25     62,09%
         26     61,77%
         27     61,51%
         28     61,01%
         29     60,09%
         30     59,06%
         31     58,36%
         32     57,51%
         33     56,70%
         34     55,28%
         35     53,73%
         36     49,77%
         37     40,90%
         38     34,85%
